# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Brandon Alcaide 
Date: 8/18/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [45]:
import os, json, time, datetime as dt, csv, pathlib
import yfinance as yf
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [46]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [48]:
SYMBOL = "MSFT"
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
print(df_api)
df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

/var/folders/1s/4hrg18b5123428pg9nyyzk3c0000gn/T/ipykernel_42667/667438490.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
[*********************100%***********************]  1 of 1 completed

Price        Date       Close
Ticker                   MSFT
0      2025-05-19  458.869995
1      2025-05-20  458.170013
2      2025-05-21  452.570007
3      2025-05-22  454.859985
4      2025-05-23  450.179993
..            ...         ...
58     2025-08-12  529.239990
59     2025-08-13  520.580017
60     2025-08-14  522.479980
61     2025-08-15  520.169983
62     2025-08-18  517.099976

[63 rows x 2 columns]


{'missing': [], 'shape': (63, 2), 'na_total': 0}

In [49]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-alpha_symbol-MSFT_20250818-205935.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [63]:
SCRAPE_URL = 'https://www.statmuse.com/nba/player/kobe-bryant-480/career-stats'  # TODO: replace with permitted page
print(SCRAPE_URL)
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

https://www.statmuse.com/nba/player/kobe-bryant-480/career-stats


{'missing': [], 'shape': (76, 24), 'na_total': 702}

In [64]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250818-211304.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.